# **Catégorisez automatiquement des questions**

## partie 1/8 : analyse exploratoire

### <br> Notebook d’exploration et de pré-traitement des questions, comprenant une analyse univariée et multivariée, un nettoyage des questions, un feature engineering de type bag of words avec réduction de dimension (du vocabulaire et des tags) 

<br>


## 1 Objectifs, imports


### 1.1 Contexte


In [1]:
# Nous avons téléchargé 50 000 questions posées sur stack overflow,
# ainsi que les tags associés.

# Objectifs de notebook :

# Comprendre nos données (forme, structure)
# premier nettoyage -> reduction dim
# Avoir une première représentation de nos données
# Rechercher des patterns
# se faire une idée de la difficulté de la tâche (prédiction de tags)
# (encore et surtout) : transformations en vue de reduction dim

# => traitement de strings avec nltk, regex, etc..
# tester spacy

# Dans le cadre de ce projet :
# Nous avons en input un fichier csv contenant 50 000 titres, questions (body), tags, etc...
# Nous avons besoin en output de représentations spécifiques, pour nos différents modèles.

# Pour les modèles non-supervisé, il nous faut :
# - un bag of words pour la LDA (/algos similaires)
# - un TS-IDF pour la NMF
# Dans les deux cas, cela implique un cleaning "aggressif", visant à conserver uniquement
# les mots qui ont du sens pour ce projet (= qui aident la prédiction de tag par les modèles).
# encore et tjs, la reduction dim !

# / supervisés ?


### 1.2 Importation des librairies, réglages


In [2]:
import os, sys, random
from zipfile import ZipFile
import numpy as np
import pandas as pd

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from wordcloud import WordCloud
from PIL import Image

# NLP
from bs4 import BeautifulSoup
import re, string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from collections import Counter
# if necessary
# !python -m spacy download en_core_web_sm
# !python -m spacy download en_core_web_md
import spacy

from pandarallel import pandarallel

print('\nPython version ' + sys.version)
print('pandas version ' + pd.__version__)
print('sns version ' + sns.__version__)

plt.style.use('ggplot')
pd.set_option('display.max_columns', 200)
sns.set(font_scale=1)

# Make sure we downloaded the models successfully
nlp = spacy.load("en_core_web_sm")
nlp = spacy.load("en_core_web_md")

num_cores = os.cpu_count()
print(f"\nNumber of CPU cores: {num_cores}")

pandarallel.initialize(progress_bar=False, nb_workers=6)


[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
2023-12-06 18:23:55.795089: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.



Python version 3.11.4 (main, Jul  5 2023, 14:15:25) [GCC 11.2.0]
pandas version 2.1.1
sns version 0.12.2

Number of CPU cores: 8
INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


### 1.3 Fonctions


In [3]:
# Principalement des fonctions pour l'affichage des graphiques pdt l'EDA

def quick_look(df, miss=True):
    """
    Display a quick overview of a DataFrame, including shape, head, tail, unique values, and duplicates.

    Args:
        df (pandas.DataFrame): The input DataFrame to inspect.
        check_missing (bool, optional): Whether to check and display missing values (default is True).

    The function provides a summary of the DataFrame, including its shape, the first and last rows, the count of unique values per column, and the number of duplicates.
    If `check_missing` is set to True, it also displays missing value information.
    """
    print(f'shape : {df.shape}')

    display(df.head())
    display(df.tail())

    print('uniques :')
    display(df.nunique())

    print('Doublons ? ', df.duplicated(keep='first').sum(), '\n')

    if miss:
        display(get_missing_values(df))


def lerp(a, b, t):
    """
    Linear interpolation between two values 'a' and 'b' at a parameter 't'.
    A very useful little function, used here to position annotations in plots.
    Got it coding with Radu :)

    Given two values 'a' and 'b', and a parameter 't',
    this function calculates the linear interpolation between 'a' and 'b' at 't'.

    Parameters:
    a (float or int): The start value.
    b (float or int): The end value.
    t (float): The interpolation parameter (typically in the range [0, 1], but can be outside).

    Returns:
    float or int: The interpolated value at parameter 't'.
    """
    return a + (b - a) * t


def generate_random_pastel_colors(n):
    """
    Generates a list of n random pastel colors, represented as RGBA tuples.

    Parameters:
    n (int): The number of pastel colors to generate.

    Returns:
    list: A list of RGBA tuples representing random pastel colors.

    Example:
    >>> generate_random_pastel_colors(2)
    [(0.749, 0.827, 0.886, 1.0), (0.886, 0.749, 0.827, 1.0)]
    """
    colors = []
    for _ in range(n):
        # Generate random pastels
        red = round(random.randint(150, 250) / 255.0, 3)
        green = round(random.randint(150, 250) / 255.0, 3)
        blue = round(random.randint(150, 250) / 255.0, 3)

        # Create an RGB color tuple and add it to the list
        color = (red,green,blue, 1.0)
        colors.append(color)

    return colors

print(generate_random_pastel_colors(2))


def get_missing_values(df):
    """Generates a DataFrame containing the count and proportion of missing values for each feature.

    Args:
        df (pandas.DataFrame): The input DataFrame to analyze.

    Returns:
        pandas.DataFrame: A DataFrame with columns for the feature name, count of missing values,
        count of non-missing values, proportion of missing values, and data type for each feature.
    """
    # Count the missing values for each column
    missing = df.isna().sum()

    # Calculate the percentage of missing values
    percent_missing = df.isna().mean() * 100

    # Create a DataFrame to store the results
    missings_df = pd.DataFrame({
        'column_name': df.columns,
        'missing': missing,
        'present': df.shape[0] - missing,  # Count of non-missing values
        'percent_missing': percent_missing.round(2),  # Rounded to 2 decimal places
        'type': df.dtypes
    })

    # Sort the DataFrame by the count of missing values
    missings_df.sort_values('missing', inplace=True)

    return missings_df

# with pd.option_context('display.max_rows', 1000):
#   display(get_missing_values(df))


def hist_distrib(dataframe, feature, bins, decimal_places, density=True):
    """
    Visualize the empirical distribution of a numerical feature using a histogram.
    Calcul des principaux indicateurs de tendance centrale, dispersion et forme.

    Args:
        dataframe (pandas.DataFrame): The input DataFrame containing the feature.
        feature (str): The name of the numerical feature to visualize.
        bins (int): The number of bins for the histogram.
        decimal_places (int): The number of decimal places for rounding numeric values.
        density (bool, optional): Whether to display the histogram as a density plot (default is True).

    Returns:
        float: The skewness of the feature's distribution.

    The function generates a histogram of the feature, displays various statistics, and returns the skewness of the distribution.
    """
    # Calculate central tendencies and dispersion
    mode_value = round(dataframe[feature].mode()[0], decimal_places)
    mode_non_zero = "N/A"
    if (dataframe[feature] != 0).any():
        mode_non_zero = round(dataframe.loc[dataframe[feature] != 0, feature].mode()[0], decimal_places)
    median_value = round(dataframe[feature].median(), decimal_places)
    mean_value = round(dataframe[feature].mean(), decimal_places)

    # Calculate dispersion
    var_emp = round(dataframe[feature].var(ddof=0), decimal_places)
    coeff_var = round(dataframe[feature].std(ddof=0), decimal_places)

    # Calculate shape indicators
    skewness_value = round(dataframe[feature].skew(), 2)
    kurtosis_value = round(dataframe[feature].kurtosis(), 2)

    # Create the plot
    fig, ax = plt.subplots(figsize=(12, 5))
    dataframe[feature].hist(density=density, bins=bins, ax=ax)

    # Adjust placement for annotations
    yt = plt.yticks()
    y_position = lerp(yt[0][0], yt[0][-1], 0.8)
    y_increment = y_position / 20
    xt = plt.xticks()
    x_position = lerp(xt[0][0], xt[0][-1], 0.7)

    # Add annotations with horizontal and vertical alignment
    annotation_fs = 13
    color = 'g'
    ax.annotate(f'Mode: {mode_value}', xy=(x_position, y_position), fontsize=annotation_fs,
                xytext=(x_position, y_position), color=color, ha='left', va='bottom')
    ax.annotate(f'Mode +: {mode_non_zero}', xy=(x_position, y_position - y_increment), fontsize=annotation_fs,
                xytext=(x_position, y_position - y_increment), color=color, ha='left', va='bottom')
    ax.annotate(f'Median: {median_value}', xy=(x_position, y_position - 2 * y_increment), fontsize=annotation_fs,
                xytext=(x_position, y_position - 2 * y_increment), color=color, ha='left', va='bottom')
    ax.annotate(f'Mean: {mean_value}', xy=(x_position, y_position - 3 * y_increment), fontsize=annotation_fs,
                xytext=(x_position, y_position - 3 * y_increment), color=color, ha='left', va='bottom')
    ax.annotate(f'Var Emp: {var_emp}', xy=(x_position, y_position - 5 * y_increment), fontsize=annotation_fs,
                xytext=(x_position, y_position - 5 * y_increment), color=color, ha='left', va='bottom')
    ax.annotate(f'Coeff Var: {coeff_var}', xy=(x_position, y_position - 6 * y_increment), fontsize=annotation_fs,
                xytext=(x_position, y_position - 6 * y_increment), color=color, ha='left', va='bottom')
    ax.annotate(f'Skewness: {skewness_value}', xy=(x_position, y_position - 8 * y_increment), fontsize=annotation_fs,
                xytext=(x_position, y_position - 8 * y_increment), color=color, ha='left', va='bottom')
    ax.annotate(f'Kurtosis: {kurtosis_value}', xy=(x_position, y_position - 9 * y_increment), fontsize=annotation_fs,
                xytext=(x_position, y_position - 9 * y_increment), color=color, ha='left', va='bottom')

    # Label the x-axis and y-axis
    ax.set_xlabel(feature, fontsize=12)
    ax.set_ylabel('Frequency', fontsize=12)

    # Show the plot
    plt.title(f'Distribution of {feature}', pad=20, fontsize=18)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.show()

    return skewness_value


def boxplot_distrib(dataframe, feature):
    """
    Affiche un boxplot, pour visualiser les tendances centrales et la dispersion d'une variable.

    Args:
        dataframe (pandas.DataFrame): The input DataFrame containing the feature.
        feature (str): The name of the numerical feature to visualize.

    The function generates a box plot of the feature to display central tendencies (median and mean) and dispersion.
    """
    fig, ax = plt.subplots(figsize=(10, 4))

    medianprops = {'color':"blue"}
    meanprops = {'marker':'o', 'markeredgecolor':'black',
            'markerfacecolor':'firebrick'}

    dataframe.boxplot(feature, vert=False, showfliers=False, medianprops=medianprops, patch_artist=True, showmeans=True, meanprops=meanprops)

    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.show()


def courbe_lorenz(dataframe, feature):
    """
    Affiche une courbe de Lorenz, pour visualiser la concentration d'une variable
    Calcule l'indice de Gini
    Visualize a Lorenz curve to assess the concentration of a variable and calculate the Gini coefficient.

    Args:
        dataframe (pandas.DataFrame): The input DataFrame containing the feature.
        feature (str): The name of the numerical feature to visualize.

    The function generates a Lorenz curve to assess the concentration of the feature and calculates the Gini coefficient.
    """
    fig, ax = plt.subplots(figsize=(12, 5))
    values = dataframe.loc[dataframe[feature].notna(), feature].values
    # print(values)
    n = len(values)
    lorenz = np.cumsum(np.sort(values)) / values.sum()
    lorenz = np.append([0],lorenz) # La courbe de Lorenz commence à 0

    xaxis = np.linspace(0-1/n,1+1/n,n+1)
    #Il y a un segment de taille n pour chaque individu, plus 1 segment supplémentaire d'ordonnée 0.
    # #Le premier segment commence à 0-1/n, et le dernier termine à 1+1/n.
    plt.plot(xaxis,lorenz,drawstyle='steps-post')
    plt.plot(np.arange(2),[x for x in np.arange(2)])
    # calcul de l'indice de Gini
    AUC = (lorenz.sum() -lorenz[-1]/2 -lorenz[0]/2)/n # Surface sous la courbe de Lorenz. Le premier segment (lorenz[0]) est à moitié en dessous de 0, on le coupe donc en 2, on fait de même pour le dernier segment lorenz[-1] qui est à moitié au dessus de 1.
    S = 0.5 - AUC # surface entre la première bissectrice et le courbe de Lorenz
    gini = 2*S
    plt.annotate('gini =  ' + str(round(gini, 2)), xy = (0.04, 0.88), fontsize = 13, xytext = (0.04, 0.88), color = 'g')
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.show()


def graphs_analyse_uni(dataframe, feature, bins=50, r=5, density=True):
    """
    Affiche histogramme + boxplot + courbe de Lorenz

    Args:
        dataframe (pandas.DataFrame): The input DataFrame containing the feature.
        feature (str): The name of the numerical feature to analyze.
        bins (int, optional): The number of bins for the histogram (default is 50).
        decimal_places (int, optional): The number of decimal places for rounding numeric values (default is 5).
        density (bool, optional): Whether to display the histogram as a density plot (default is True).

    The function generates and displays an analysis of the given numerical feature, including an histogram, a box plot, and a Lorenz curve.
    """
    hist_distrib(dataframe, feature, bins, r)
    boxplot_distrib(dataframe, feature)
    courbe_lorenz(dataframe, feature)


def shape_head(df, nb_rows=5):
    """
    Affiche les dimensions et les premières lignes dùun dataframe
    Display the dimensions and the first rows of a DataFrame.

    Args:
        df (pandas.DataFrame): The input DataFrame to display.
        nb_rows (int, optional): The number of rows to display (default is 5, max is 60).

    The function prints the dimensions of the DataFrame and displays the first few rows.
    """
    print(df.shape)
    display(df.head(nb_rows))


def doughnut(df, feature, title, width=10, height=10):
    """
    Affiche la répartition d'une feature sous forme de diagramme circulaire
    Display the distribution of a feature as a doughnut chart.
    Les couleurs sont aléatoires.

    Args:
        df (pandas.DataFrame): The input DataFrame containing the feature.
        feature (str): The name of the feature to visualize.
        title (str): The title for the doughnut chart.
        width (int, optional): The width of the chart (default is 10).
        height (int, optional): The height of the chart (default is 10).

    The function creates a doughnut chart to visualize the distribution of the specified feature.
    If you don't like the colors, try running it again :)
    """
    colors = generate_random_pastel_colors(20)

    grouped_df = df.groupby(feature).size().to_frame("count_per_type").reset_index()
    pie = grouped_df.set_index(feature).copy()

    fig, ax = plt.subplots(figsize=(width, height))

    patches, texts, autotexts = plt.pie(x=pie['count_per_type'], autopct='%1.1f%%',
        startangle=-30, labels=pie.index, textprops={'fontsize':11, 'color':'#000'},
        labeldistance=1.25, pctdistance=0.85, colors=colors)

    plt.title(
    label=title,
    fontdict={"fontsize":17},
    pad=20
    )

    for text in texts:
        # text.set_fontweight('bold')
        text.set_horizontalalignment('center')

    # Customize percent labels
    for autotext in autotexts:
        autotext.set_horizontalalignment('center')
        autotext.set_fontstyle('italic')
        autotext.set_fontsize('10')

    #draw circle
    centre_circle = plt.Circle((0,0),0.7,fc='white')
    fig = plt.gcf()
    fig.gca().add_artist(centre_circle)

    plt.show()


def get_non_null_values(df):
    """
    Génère un dataframe contenant le nombre et la proportion de non-null (non-zero) valeurs pour chaque feature
    Generate a DataFrame containing the count and proportion of non-null (non-zero) values for each feature.

    Args:
        df (pandas.DataFrame): The input DataFrame to analyze.

    The function calculates and returns a DataFrame with the count and percentage of non-null (non-zero) values for each feature.
    """
    non_null_counts = df.ne(0).sum()
    percent_non_null = (non_null_counts / df.shape[0]) * 100
    non_null_values_df = pd.DataFrame({'column_name': df.columns,
                                       'non_null_count': non_null_counts,
                                       'percent_non_null': percent_non_null.round(2),
                                       'type': df.dtypes})
    non_null_values_df.sort_values('non_null_count', inplace=True)
    return non_null_values_df


def get_colors(n=7):
    """
    Generate a list of random colors from multiple colormaps.

    Args:
        n (int, optional): The number of colors to sample from each colormap (default is 7).

    Returns:
        list: A list of random colors sampled from different colormaps.
    """
    num_colors_per_colormap = n
    colormaps = [plt.cm.Pastel2, plt.cm.Set1, plt.cm.Paired]
    all_colors = []

    for colormap in colormaps:
        colors = colormap(np.linspace(0, 1, num_colors_per_colormap))
        all_colors.extend(colors)

    np.random.shuffle(all_colors)

    return all_colors


[(0.745, 0.627, 0.949, 1.0), (0.929, 0.808, 0.855, 1.0)]


## 2 EDA et preprocessing des questions


### 2.1 Importation des données brutes


In [4]:
# Donnees compressées sinon on dépasse la limite /objet de Github (50Mb)

# path to the zip file
zip_file_path = './../data/raw_data/QueryResults.zip'

# directory where you want to extract the contents
extract_to_dir = './../data/raw_data'

# Open the zip file
with ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all the contents into the specified directory
    zip_ref.extractall(extract_to_dir)

# L'encodage est bien UTF-8 (vérifié en ouvrant le .csv ds vscode)
raw_data = pd.read_csv('./../data/raw_data/QueryResults.csv', sep=',')

quick_look(raw_data)


shape : (50000, 9)


,Title,Body,Tags,Id,Score,ViewCount,FavoriteCount,AnswerCount,CreationDate
0,ImportError: cannot import name 'url_decode' f...,<p>I am building a webapp using Flask. I impor...,<python><flask><importerror><flask-login><werk...,77215107,13,14443,NaN,5,2023-10-02 11:07:45
1,Compilation error after upgrading to JDK 21 - ...,"<p>After upgrading to JDK 21, I have the follo...",<spring-boot><compiler-errors><upgrade><lombok...,77171270,55,36788,NaN,3,2023-09-25 09:05:11
2,Differences between Langchain & LlamaIndex,<p>I'm currently working on developing a chatb...,<chatbot><openai-api><langchain><large-languag...,76990736,28,10433,NaN,2,2023-08-28 07:22:32
3,session not created: This version of ChromeDri...,<p>I am running a Docker image from a Docker c...,<python><amazon-web-services><docker><google-c...,76909437,14,14969,NaN,8,2023-08-15 22:21:03
4,Spring security method cannot decide pattern i...,<p>When I try to run an application it fails t...,<java><spring-boot><eclipse><spring-security><...,76809698,27,18943,NaN,8,2023-08-01 08:16:21


,Title,Body,Tags,Id,Score,ViewCount,FavoriteCount,AnswerCount,CreationDate
49995,How can I send a file document to the printer ...,<p>Here's the basic premise:</p>\n\n<p>My user...,<c#><winforms><pdf><.net-4.0><printing>,6103705,91,215784,0.0,12,2011-05-23 22:22:56
49996,CA1014 Mark 'some.dll' with CLSCompliant(true)...,"<p>When I run StyleCop, I got this error messa...",<visual-studio><visual-studio-2010><dll><style...,6103133,17,11024,0.0,2,2011-05-23 21:15:51
49997,How to change a text file's name in C++?,<p>I would like to change a <code>txt</code> f...,<c++><algorithm><file><directory><file-rename>,6103036,16,37118,0.0,3,2011-05-23 21:05:59
49998,php implode (101) with quotes,<p>Imploding a simple array </p>\n\n<p>would ...,<php><arrays><string><csv><implode>,6102398,156,141141,0.0,16,2011-05-23 20:06:35
49999,What characters are allowed in a iOS file name?,<p>I'm looking for a way to make sure a string...,<ios><file><filenames><character-encoding><nsf...,6102333,29,26085,0.0,10,2011-05-23 20:00:57


uniques :


Title            49999
Body             50000
Tags             48252
Id               50000
Score              761
ViewCount        36831
FavoriteCount        2
AnswerCount         64
CreationDate     49994
dtype: int64

Doublons ?  0 



,column_name,missing,present,percent_missing,type
Title,Title,0,50000,0.00,object
Body,Body,0,50000,0.00,object
Tags,Tags,0,50000,0.00,object
Id,Id,0,50000,0.00,int64
Score,Score,0,50000,0.00,int64
ViewCount,ViewCount,0,50000,0.00,int64
AnswerCount,AnswerCount,0,50000,0.00,int64
CreationDate,CreationDate,0,50000,0.00,object
FavoriteCount,FavoriteCount,1827,48173,3.65,float64


In [5]:
# Pas de valeurs manquantes, à part FavoriteCount (pas important)

# Seulement 2 titres identiques / 50 000 lignes
# id est bien une clé primaire
# body aussi, pas de questions en doublon (identiques)

# On a environ 48 000 sets de tags différents, vérifier + tard combien de tags uniques

# Les types semblent corrects,
# à part les dates bien sûr

raw_data['CreationDate'] = pd.to_datetime(raw_data['CreationDate'])
# Après je pense qu'on n'utilisera jamais ces dates... Juste au cas où.

raw_data.describe()

# Avec nos critères (cf requete sql, fin ntbk2), il a fallu retourner jusqu'à mai 2011
# pour avoir 50 000 questions. On retrouve ici le favoriteCount très bas, proche de 0.
# On a une moyenne pour le score (environ 50), le nv de vues (66 000) et de réponses (>5)
# dans notre corpus.


,Id,Score,ViewCount,FavoriteCount,AnswerCount,CreationDate
count,5.000000e+04,50000.000000,5.000000e+04,48173.000000,50000.000000,50000
mean,2.878203e+07,49.973980,6.612881e+04,0.000083,5.384480,2015-02-23 09:48:58.933900032
min,6.102333e+06,11.000000,1.000200e+04,0.000000,2.000000,2011-05-23 20:00:57
25%,1.413152e+07,15.000000,1.842050e+04,0.000000,3.000000,2013-01-03 00:42:30.500000
50%,2.551682e+07,24.000000,3.210600e+04,0.000000,4.000000,2014-08-26 23:41:22
75%,4.133835e+07,45.000000,6.446725e+04,0.000000,6.000000,2016-12-27 02:23:45
max,7.721511e+07,27153.000000,1.063924e+07,1.000000,87.000000,2023-10-02 11:07:45
std,1.687404e+07,168.429881,1.424566e+05,0.009112,4.622519,NaN


In [6]:
raw_questions_tags = raw_data[['Title', 'Body', 'Tags']].copy()

# Rename columns
raw_questions_tags = raw_questions_tags.rename(columns={'Title': 'title',
                                                        'Body': 'body',
                                                        'Tags': 'tags'})


### 2.2 Suppression des tags html


In [7]:
raw_questions_tags['body'] = raw_questions_tags['body'].parallel_apply(
    lambda x: BeautifulSoup(x, 'html.parser').get_text())

quick_look(raw_questions_tags)

# environ 25s sans pandarallel -> 5-6s sur 6 cores
# pandarallel adopté !


shape : (50000, 3)


,title,body,tags
0,ImportError: cannot import name 'url_decode' f...,I am building a webapp using Flask. I imported...,<python><flask><importerror><flask-login><werk...
1,Compilation error after upgrading to JDK 21 - ...,"After upgrading to JDK 21, I have the followin...",<spring-boot><compiler-errors><upgrade><lombok...
2,Differences between Langchain & LlamaIndex,I'm currently working on developing a chatbot ...,<chatbot><openai-api><langchain><large-languag...
3,session not created: This version of ChromeDri...,I am running a Docker image from a Docker cont...,<python><amazon-web-services><docker><google-c...
4,Spring security method cannot decide pattern i...,When I try to run an application it fails to s...,<java><spring-boot><eclipse><spring-security><...


,title,body,tags
49995,How can I send a file document to the printer ...,Here's the basic premise:\nMy user clicks some...,<c#><winforms><pdf><.net-4.0><printing>
49996,CA1014 Mark 'some.dll' with CLSCompliant(true)...,"When I run StyleCop, I got this error message ...",<visual-studio><visual-studio-2010><dll><style...
49997,How to change a text file's name in C++?,"I would like to change a txt file's name, but ...",<c++><algorithm><file><directory><file-rename>
49998,php implode (101) with quotes,Imploding a simple array \nwould look like th...,<php><arrays><string><csv><implode>
49999,What characters are allowed in a iOS file name?,I'm looking for a way to make sure a string ca...,<ios><file><filenames><character-encoding><nsf...


uniques :


title    49999
body     50000
tags     48252
dtype: int64

Doublons ?  0 



,column_name,missing,present,percent_missing,type
title,title,0,50000,0.0,object
body,body,0,50000,0.0,object
tags,tags,0,50000,0.0,object


In [8]:
# Je ne vois pas de balises dans les titres
# mais j'avoue je n'ai pas lu les 50 000 titres, donc au cas où :

raw_questions_tags['title'] = raw_questions_tags['title'].parallel_apply(
    lambda x: BeautifulSoup(x, 'html.parser').get_text())

# Le warning n'a pas d'importance ici : du texte qui ne contient pas de tags html
# n'est pas modifié par BeautifulSoup.get_text()

# J'hésitais à concaténer title + body ici, mais on testera les modeles seulment sur les titres
# pour avoir une baseline. On va donc conserver les 2 features, et les traiter en parallèle.


/tmp/ipykernel_407073/1392450844.py:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  lambda x: BeautifulSoup(x, 'html.parser').get_text())
/tmp/ipykernel_407073/1392450844.py:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  lambda x: BeautifulSoup(x, 'html.parser').get_text())
/tmp/ipykernel_407073/1392450844.py:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  lambda x: BeautifulSoup(x, 'html.parser').get_text())
/tmp/ipykernel_407073/1392450844.py:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  lambda x: BeautifulSoup(x, 'html.parser').get_

### 2.3 Tokenisation, majuscules, ponctuation


In [9]:
# Après qq tests, le + efficace ici semble d'appliquer d'abord la fonction word_tokenize(),
# qui gère mieux beaucoup de spécificités de l'anglais (ex: contractions),
# puis le RegexpTokenizer pour 'découper' ensuite les adresses mails, URLS, etc...
# Il y aurait d'autres stratégies possibles, par exemples utiliser d'abord le module contractions.

def preprocess_1_tokenize(text, remove_punct=True):
    """
    Clean input text (lower, strip spaces), Tokenize, Remove punctuation.

    Parameters:
    - text (str): Input text to tokenize.
    - remove_punct (bool): filter punctuation marks.

    Returns:
    - list: List of tokens.
    """
    # Nettoyage des majuscules et des espaces
    text = text.lower().strip()
    try:
        tokens = nltk.word_tokenize(text)
        tokenizer = nltk.RegexpTokenizer(r'\w+')
        tokens = tokenizer.tokenize(" ".join(tokens))  # Apply RegexpTokenizer to the entire list

        if remove_punct:
            # Remove punctuation (make sure)
            tokens = [token for token in tokens if token not in string.punctuation]

    except Exception as e:
        print(f"Error in tokenization: {e}")
        return []

    return tokens


raw_questions_tags['body_tokens'] = raw_questions_tags['body'].parallel_apply(
    lambda x: preprocess_1_tokenize(x))


In [10]:
# Idem for the 'title' column
raw_questions_tags['title_tokens'] = raw_questions_tags['title'].parallel_apply(
    lambda x: preprocess_1_tokenize(x))


In [14]:
raw_questions_tags['nb_tokens'] = len(set(raw_questions_tags['title_tokens']
                                           + raw_questions_tags['body_tokens']))

display(raw_questions_tags)


TypeError: unhashable type: 'list'

### 2.4 tokens uniques


In [ ]:
def preprocess_2_keep_uniques_only(liste_tokens):
    seen_tokens = set()
    unique_tokens = []

    for token in liste_tokens:
        if token not in seen_tokens:
            seen_tokens.add(token)
            unique_tokens.append(token)

    return unique_tokens

# Apply the preprocessing function to body
raw_questions_tags['body_tokens_uniques'] = raw_questions_tags['body_tokens'].parallel_apply(preprocess_2_keep_uniques_only)
print('Done')


In [ ]:
# and title
raw_questions_tags['title_tokens_uniques'] = raw_questions_tags['title_tokens'].parallel_apply(preprocess_2_keep_uniques_only)
print('Done')


In [ ]:
raw_questions_tags['nb_tokens_uniques'] = len(set((raw_questions_tags['title_tokens_uniques'].parallel_apply(
    lambda x: preprocess_1_tokenize(x)) + raw_questions_tags['body_tokens_uniques'].parallel_apply(
    lambda x: preprocess_1_tokenize(x)))))

display(raw_questions_tags)


In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

# Plotting 'nb_mots'
raw_questions_tags['nb_tokens'].hist(density=False, bins=50, ax=ax, color='#7cf', alpha=0.6, label='nb_mots')

# Plotting 'nb_mots_uniques'
raw_questions_tags['nb_tokens_uniques'].hist(density=False, bins=50, ax=ax, label='nb_mots_uniques')

plt.title('Vocabulaire', pad=20, fontsize=18)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

ax.legend()

plt.show()


In [ ]:
graphs_analyse_uni(raw_questions_tags, 'nb_tokens', bins=50, r=2, density=True)


In [ ]:
graphs_analyse_uni(raw_questions_tags, 'nb_tokens_uniques', bins=50, r=2, density=True)

# moyenne divisée par 2


### 2.5 Fréquence des mots


In [ ]:
# Extract tokens from the 'tokens_uniques' column
all_tokens = set([token for tokens_list in raw_questions_tags['title_tokens_uniques'] for token in tokens_list]
                 + [token for tokens_list in raw_questions_tags['body_tokens_uniques'] for token in tokens_list])

print (len(all_tokens))

# Calculate token frequencies using a loop
# J'ai découvert plus tard la fonction FreqDist() de nltk, qui fait la mm chose
token_frequencies_dict = {}
for token in all_tokens:
    token_frequencies_dict[token] = token_frequencies_dict.get(token, 0) + 1

# Display the first 50 items in the token frequencies dictionary
for token, frequency in list(token_frequencies_dict.items())[:50]:
    print(f"{token}: {frequency}")


In [ ]:
# Sort the dictionary items by values in descending order
sorted_token_frequencies = dict(sorted(token_frequencies_dict.items(), key=lambda item: item[1], reverse=True))

# Display the first 50 items in the token frequencies dictionary
for token, frequency in list(sorted_token_frequencies.items())[:50]:
    print(f"{token}: {frequency}")

# La plupart de ces mots appartiennent probablement à la liste des stopwords


In [ ]:
# Visualisons

df_freq = pd.DataFrame(list(token_frequencies_dict.items()), columns=['Token', 'Frequency'])

# Sort the DataFrame by frequency in descending order
df_freq = df_freq.sort_values(by='Frequency', ascending=False)


In [ ]:
# Plot the distribution using Plotly

df = df_freq[:200]

fig = px.bar(df, x='Token', y='Frequency', labels={'Token': 'Token', 'Frequency': 'Frequency'},
             title='Token Frequency Distribution', text='Frequency')
fig.update_xaxes(tickangle=45, tickmode='array', tickvals=df.index, ticktext=df['Token'])

# Show the plot
fig.show()


### 2.6 Stopwords


In [ ]:
# Get a set of English stop words from NLTK
stopwords = set(nltk.corpus.stopwords.words('english'))

# Update the set with additional stop words if needed (most frequent ?)
# stopwords.update(['additional', 'stop', 'words'])

important_tokens = [token for token in all_tokens if token not in stopwords]
print(len(important_tokens))

# Display the first 50 items in the token frequencies dictionary
for token, frequency in list(sorted_token_frequencies.items())[:200]:
    if token in important_tokens:
        print(f"{token}: {frequency}")

# Il y a encore bcp de mots très communs. Pas sûr où placer le seuil


In [ ]:
def preprocess_text_3(liste_tokens):
    # ajouter /frequence ici ?

    filtered_list = [token for token in liste_tokens if token not in stopwords]

    return filtered_list

# Apply the preprocessing function
raw_questions_tags['tokens_uniques_no_stopwords'] = raw_questions_tags['tokens_uniques_regex'].parallel_apply(preprocess_text_3)
print('Done')


In [ ]:
raw_questions_tags['nb_tokens_uniques_no_stopwords'] = raw_questions_tags['tokens_uniques_no_stopwords'].parallel_apply(len)
print('Done')


In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

# Plotting 'nb_mots'
raw_questions_tags['nb_mots_regex'].hist(density=False, bins=50, ax=ax, color='#7cf', alpha=0.6, label='nb_mots')

# Plotting 'nb_mots_uniques'
raw_questions_tags['nb_mots_uniques_regex'].hist(density=False, bins=50, ax=ax, label='nb_mots_uniques')

# Plotting 'nb_mots_uniques non stopwords'
raw_questions_tags['nb_tokens_uniques_no_stopwords'].hist(density=False, bins=50, color='#fc3', ax=ax,
                                                          label='nb_mots_uniques hors stopwords')

plt.title('Vocabulaire', pad=20, fontsize=18)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

ax.legend()

plt.show()


### 2.7 Lemmatization + filtrage syntaxique


In [ ]:
# Lemmatization tends to be more accurate but can be slower than stemming.
# On a utilisé le stemming en français, dans le cours openclassrooms
# Utilisons la lemmatisation ici, + appropriée pour notre objectif
# (extraire le sens important, le thème, pour proposer des tags)
# On peut aussi profiter de l'analyse grammaticale (POS) pour
# conserver uniquement les mots les plus "importants" dans la phrase, les noms et les verbes.

def lemmatize_tokens(tokens, filter=True):
    lemmatizer = WordNetLemmatizer()

    def get_wordnet_pos(pos_tag):
        if pos_tag.startswith('J'):
            return wordnet.ADJ
        elif pos_tag.startswith('R'):
            return wordnet.ADV
        elif pos_tag.startswith('V'):
            return wordnet.VERB
        elif pos_tag.startswith('N'):
            return wordnet.NOUN
        else:
            return wordnet.NOUN  # Default to noun if the part of speech is not recognized

    # Get part of speech for each token
    pos_tags = nltk.pos_tag(tokens)

    # Lemmatize each token
    lemmatized_tokens = []
    for token, pos_tag in pos_tags:
        if filter and pos_tag.startswith(('J', 'R')):
            # Exclude adjectives (J) and adverbs (R) if filter is enabled
            lemmatized_tokens.append(" ")
        else:
            lemmatized_tokens.append(lemmatizer.lemmatize(token, pos=get_wordnet_pos(pos_tag)))

    return lemmatized_tokens

# Assuming raw_questions_tags is a DataFrame with a column 'tokens_uniques_no_stopwords'
# Replace 'tokens_uniques_no_stopwords' with the actual column name if needed
raw_questions_tags['lemms'] = raw_questions_tags['tokens_uniques_no_stopwords'].parallel_apply(lemmatize_tokens)
print('Done')


In [ ]:
# garde seulmt les lemmes uniques
raw_questions_tags['lemms_uniques'] = raw_questions_tags['lemms'].parallel_apply(preprocess_2_keep_uniques_only)
print('Done')


In [ ]:
raw_questions_tags['nb_lemms'] = raw_questions_tags['lemms'].parallel_apply(len)
raw_questions_tags['nb_lemms_uniques'] = raw_questions_tags['lemms_uniques'].parallel_apply(len)
print('Done')


In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

# Plotting 'nb_mots'
raw_questions_tags['nb_mots_regex'].hist(density=False, bins=50, ax=ax, color='#7cf', alpha=0.6, label='nb_mots')

# Plotting 'nb_mots_uniques'
raw_questions_tags['nb_mots_uniques_regex'].hist(density=False, bins=50, ax=ax, label='nb_mots_uniques')

# Plotting 'nb_mots_uniques non stopwords'
raw_questions_tags['nb_tokens_uniques_no_stopwords'].hist(density=False, bins=50, color='#fc3', ax=ax,
                                                          label='nb_mots_uniques hors stopwords')
# Plotting 'lemms'
raw_questions_tags['nb_lemms_uniques'].hist(density=False, bins=50, color='#f81', ax=ax,
                                                          label='nb_lemms_uniques')

plt.title('Vocabulaire', pad=20, fontsize=18)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

ax.legend()

plt.show()

# On voit une petite différence


### 2.8 Explorer le corpus obtenu


In [ ]:
# Affichons 20 questions au hasard (+ les tags)

# display(raw_questions_tags[['questions', 'lemms']].sample(20))

display(raw_questions_tags.sample(20))


In [ ]:
# Pistes restantes

# Handling Numeric Values:
# Decide whether to keep or remove numeric values based on their relevance to your prediction task.
# Rare Words: Consider removing or replacing rare words to reduce the dimensionality of your data.
# Contractions: Expand contractions to ensure consistency (e.g., "don't" to "do not").
# URLs and Email Addresses

# Pour les tags
# Label Encoding: If your tags are categorical, encode them into numerical labels.


## 3 Target


### 3.1 Preprocessing


In [ ]:
def preprocess_tags(tags_string):
    # just in case
    tags_string = tags_string.lower()
    # Split tags by '><'
    tags_list = tags_string.split('><')

    # Remove angle brackets from the first and last tag
    tags_list[0] = tags_list[0][1:]
    tags_list[-1] = tags_list[-1][:-1]

    return tags_list

# Apply the preprocessing function to the 'tags' column
raw_questions_tags['tags_list'] = raw_questions_tags['tags'].apply(preprocess_tags)

display(raw_questions_tags[['tags', 'tags_list']].head(10))


In [ ]:
all_tags = [tag for tags in raw_questions_tags['tags_list'] for tag in tags]
print(len(all_tags))

# Au cas où il y aurait des doublons ds les tags
raw_questions_tags['tags_uniques'] = raw_questions_tags['tags'].apply(preprocess_2_keep_uniques_only)

# Fréquence
# Factoriser
tag_frequencies_dict = {}
for tag in all_tags:
    tag_frequencies_dict[tag] = tag_frequencies_dict.get(tag, 0) + 1

# Sort the dictionary items by values in descending order
sorted_tag_frequencies = dict(sorted(tag_frequencies_dict.items(), key=lambda item: item[1], reverse=True))

# Display the first 50 items in the tag frequencies dictionary
for tag, frequency in list(sorted_tag_frequencies.items())[:50]:
    print(f"{tag}: {frequency}")

# 250 000 tags (différents) !
# Ca repond a la question : est-ce qu'un humain pourrait facilement faire cette tache.
# Il faudrait déjà connaitre ts les tags possibles... Pas évident.

# Le tag le plus présent est python
# Est-ce qu'il y a plus de codeurs en python qu'en javascript ?
# Peut-être. Pas sûr.
# Est-ce qu'on va sur stack overflow plus souvent quand on fait du python ?
# Je dirais que oui. Il y a tjs de nveaux modules à découvrir en python !

# Classement intéressant !


### 3.2 Frequences des tags


In [ ]:
# Extract tags and frequencies
tags = list(sorted_tag_frequencies.keys())
frequencies = list(sorted_tag_frequencies.values())

# Plot
plt.figure(figsize=(12, 6))
plt.bar(tags[:100], frequencies[:100], color='skyblue')
plt.xlabel('Tags')
plt.ylabel('Frequency')
plt.title('Top 50 Tag Frequencies')
plt.xticks(rotation=90, ha='right', fontsize=8)  # Rotate x-axis labels for better readability
plt.show()


### 3.3 Les 20 tags les + frequents


In [ ]:
# + d'un tiers des question concernent python, js ou java.

# Faisons un top 20
df_freq = pd.DataFrame({'Tag': tags, 'Frequency': frequencies})

# Display the DataFrame
display(df_freq.head())

# plot a pie
colors = generate_random_pastel_colors(20)

fig, ax = plt.subplots(figsize=(10, 10))

patches, texts, autotexts = plt.pie(x=df_freq['Frequency'][:20], autopct='%1.1f%%',
    startangle=-30, labels=df_freq['Tag'][:20], textprops={'fontsize':11, 'color':'#000'},
    labeldistance=1.25, pctdistance=0.85, colors=colors)

plt.title(
label='20 most frequent Tags',
fontdict={"fontsize":17},
pad=20
)

for text in texts:
    # text.set_fontweight('bold')
    text.set_horizontalalignment('center')

# Customize percent labels
for autotext in autotexts:
    autotext.set_horizontalalignment('center')
    autotext.set_fontstyle('italic')
    autotext.set_fontsize('10')

#draw circle
centre_circle = plt.Circle((0,0),0.7,fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

plt.show()


### 3.4 Tags rares


In [ ]:
# A l'inverse, certains tags sont rares

# Count tags with frequency less than 10
tags_below_10 = sum(1 for frequency in sorted_tag_frequencies.values() if frequency < 10)
print(tags_below_10)

# voire tres rares
# tags hapax (1 seule occurence)

# Count tags with frequency less than 2
tags_below_2 = sum(1 for frequency in sorted_tag_frequencies.values() if frequency < 2)
print(tags_below_2)


### 3.5 wordcloud


In [ ]:
cloud = WordCloud(background_color='white',
                  stopwords=[],
                  max_words=50).generate(" ".join(all_tags))
plt.imshow(cloud)
plt.axis('off')
plt.show()

# Pourquoi python tout petit ?? C le tag le + frequent !


In [ ]:
# analyse multi ici


## Normalisation
